---

📌 **This notebook has been updated in [jhj0517/finetunings](https://github.com/jhj0517/finetunings) repository!**

---

In [ ]:
#@title #(Optional) Check GPU

#@markdown You can check your GPU setup before start.
!nvidia-smi

In [ ]:
#@title #1. Install Dependencies

!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/dataset
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

In [1]:
#@title # 2. (Optional) Mount Google Drive

#@markdown It's not mandatory but it's recommended to mount to Google Drive and use the Google Drive's path for your training image dataset.

#@markdown The dataset should have following structure:

#@markdown Each image file should have a corresponding text file (`.txt`) with the same name.
#@markdown The text file typically contains prompts or metadata associated with the image.

#@markdown ### Example File Structure:
#@markdown ```
#@markdown your-dataset/
#@markdown ├── a (1).png         # Image file
#@markdown ├── a (1).txt         # Corresponding prompt for a (1).png
#@markdown ├── a (2).png         # Another image file
#@markdown ├── a (2).txt         # Corresponding prompt for a (2).png
#@markdown ```

from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title # 3. (Optional) Register Huggingface Token To Download Base Model

#@markdown If you don't have entire base model files ([black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)) in the drive you need to sign in to Huggingface to download the model.

#@markdown Get your tokens from https://huggingface.co/settings/tokens, and register in colab's seceret as **`HF_TOKEN`** and use it in any notebook. ( 'Read' permission is enough )

#@markdown To register secrets in colab, click on the key-shaped icon in the left panel and enter your **`HF_TOKEN`** like this:

#@markdown ![image](https://miro.medium.com/v2/resize:fit:720/format:webp/1*kqKOGVkupS_R2FQ_049xLw.png))

import getpass
import os
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

#os.environ['HF_TOKEN'] = getpass.getpass(hf_token)
print("HF_TOKEN environment variable has been set.")

HF_TOKEN environment variable has been set.


In [ ]:
#@title # 4. Train with Parameters
import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

#@markdown ## Paths

#@markdown Set your dataset path and output path for lora here.
DATASET_DIR = "/content/drive/MyDrive/finetunings/dataset" # @param {type:"string"}
OUTPUT_DIR = '/content/drive/MyDrive/finetunings/trained-flux'  # @param {type:"string"}
os.makedirs(OUTPUT_DIR, exist_ok=True)

#@markdown ## Base Model
#@markdown If you'll just use the default repo id here then you need to signin to huggingface in the previous section
repo_id_or_path = 'black-forest-labs/FLUX.1-dev' # @param {type:"string"}
quantize = False # @param {type:"boolean"}
low_vram = False # @param {type:"boolean"}

#@markdown ## Process
#@markdown (max_step_saves_to_keep = how many checkpoints to keep. )
dtype = "float16" # @param {type:"string"}
save_every = 250 # @param {type:"integer"}
max_step_saves_to_keep = 4 # @param {type:"integer"}


#@markdown ## Hyper Parameters
batch_size = 1 # @param {type:"integer"}
steps = 500 # @param {type:"integer"}
gradient_accumulation_steps = 1 # @param {type:"integer"}
train_dtype = "bf16" # @param {type:"string"}
lr = 4e-4 # @param {type:"number"}



# Training
job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
        # this name will be the folder and filename name
        ('name', 'my_first_flux_lora_v1'),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                ('training_folder', OUTPUT_DIR),
                ('performance_log_every', 1000),
                ('device', 'cuda:0'),
                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', 16),
                    ('linear_alpha', 16)
                ])),
                ('save', OrderedDict([
                    ('dtype', dtype),  # precision to save
                    ('save_every', save_every),  # save every this many steps
                    ('max_step_saves_to_keep', max_step_saves_to_keep)  # how many intermittent saves to keep
                ])),
                ('datasets', [
                    OrderedDict([
                        ('folder_path', DATASET_DIR),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', 0.05),  # will drop out the caption 5% of time
                        ('shuffle_tokens', False),  # shuffle caption order, split by commas
                        ('cache_latents_to_disk', True),  # leave this true unless you know what you're doing
                        ('resolution', [512, 768, 1024])  # flux enjoys multiple resolutions
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', batch_size),
                    ('steps', steps),  # total number of steps to train 500 - 4000 is a good range
                    ('gradient_accumulation_steps', gradient_accumulation_steps),
                    ('train_unet', True),
                    ('train_text_encoder', False),  # probably won't work with flux
                    ('content_or_style', 'balanced'),  # content, style, balanced
                    ('gradient_checkpointing', True),  # need the on unless you have a ton of vram
                    ('noise_scheduler', 'flowmatch'),  # for training only
                    ('optimizer', 'adamw8bit'),
                    ('lr', lr),
                    # uncomment this to skip the pre training sample
                    #('skip_first_sample', True),

                    # ema will smooth out learning, but could slow it down. Recommended to leave on.
                    ('ema_config', OrderedDict([
                        ('use_ema', True),
                        ('ema_decay', 0.99)
                    ])),

                    # will probably need this if gpu supports it for flux, other dtypes may not work correctly
                    ('dtype', train_dtype)
                ])),
                ('model', OrderedDict([
                    # huggingface model name or path
                    ('name_or_path', repo_id_or_path),
                    ('is_flux', True),
                    ('quantize', quantize),  # run 8bit mixed precision
                    ('low_vram', low_vram),  # uncomment this if the GPU is connected to your monitors. It will use less vram to quantize, but is slower.
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),  # must match train.noise_scheduler
                    ('sample_every', 250),  # sample every this many steps
                    ('width', 1024),
                    ('height', 1024),
                    ('prompts', [
                        # Change the sample prompts as you want
                        'a futuristic cityscape at dusk, flying cars, neon lights, and a glowing sky',
                        'a woman chef cooking gourmet food in a high-tech kitchen, surrounded by advanced gadgets',
                        'a woman sitting by the fireplace in a cozy cabin, sipping tea, snowstorm visible through the window',
                        'a cowboy riding a dinosaur in the wild west, holding a lasso, with a sunset in the background',
                        'a polar bear wearing a scarf knitting in a cozy cabin, snowstorm visible through the window',
                        'a steampunk airship floating above the clouds, gears and steam visible, with people onboard',
                        'a futuristic racecar on a glass racetrack, glowing wheels, with a cyberpunk city in the background',
                        'a young woman painting a giant mural on a city wall, vibrant colors, passerby watching',
                        'a samurai standing on a mountain peak, sword drawn, cherry blossoms floating in the air',
                        'a dragon lounging in a modern living room, reading a book by the fireplace'
                    ]),
                    ('neg', ''),  # not used on flux
                    ('seed', 42),
                    ('walk_seed', True),
                    ('guidance_scale', 4),
                    ('sample_steps', 20)
                ]))
            ])
        ])
    ])),
    # you can add any additional meta info here. [name] is replaced with config name at top
    ('meta', OrderedDict([
        ('name', '[name]'),
        ('version', '1.0')
    ]))
])

run_job(job_to_run)


In [ ]:
#@title #Test Your Lora
#@markdown Once the installation is complete, you can use public URL that is displayed.

# https://huggingface.co/docs/diffusers/main/api/pipelines/flux

import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(BASE_MODEL_PATH, torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload()
pipe.load_lora_weights("black-forest-labs/FLUX.1-Canny-dev-lora")

prompt = "A cat holding a sign that says hello world"
out = pipe(
    prompt=prompt,
    guidance_scale=0.,
    height=768,
    width=1360,
    num_inference_steps=4,
    max_sequence_length=256,
).images[0]
out.save("image.png")